In [4]:
import os
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError
from dataclasses import dataclass, field

In [ ]:
# obj = query_by_address(collection, "浙江省温州市乐清市淡溪镇石龙头村1")

In [11]:
DB_NAME = "map"
COLLECTION_NAME = "amap"

In [12]:
@dataclass
class MapMongoDB:
    db_name: str  # 数据库名
    collection_name: str
    mongo_uri: str = field(
        default=os.getenv("MONGO_URI", "mongodb://localhost:27017/")
    )  # 默认本地连接)
    
    def insert_data(self, **kwargs):
        try:
            # 插入文档
            self.collection.insert_one(kwargs)
            return 1
        except DuplicateKeyError:
            return 0
        except Exception as e:
            return -1

    # query item
    def query_by_address(self, address):
        result = self.collection.find_one({"address_input": address})
        return result

    def __post_init__(self):
        """初始化数据库，创建集合并为 name 字段添加唯一索引"""
        # 连接 MongoDB
        self.client = MongoClient(self.mongo_uri)
        self.db = self.client[self.db_name]
        self.collection = self.db[self.collection_name]
        self.collection.create_index("address_input", unique=True)

    def __del__(self):
        self.client.close()

In [13]:
mongodb = MapMongoDB(db_name=DB_NAME, collection_name=COLLECTION_NAME)

In [ ]:
mongodb